In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [0]:
file_location = "/FileStore/tables/amplo2020_12.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#df.display()

In [0]:
df_lgpd = df.select("Data Abertura", "Problema")\
            .orderBy('Data Abertura')

#display(df_lgpd)

In [0]:
df_lgpd_f = df_lgpd.filter(df_lgpd.Problema.contains("dado"))

#display(df_lgpd_f)

In [0]:
df_dados = df_lgpd_f.withColumnRenamed("Problema", "Dados")

#display(df_dados)

In [0]:
df_dados_bkp = df_dados

In [0]:
df_dados = df_dados.withColumn('Dados',\
                               F.expr("coalesce(map(\
                               'elimina��o', 'eliminação',\
                               'revoga��o','revogação',\
                               'altera��o','alteração',\
                               'destrui��o','destruição',\
                               'comunica��o','comunicação',\
                               'il�cito','ilícito',\
                               'n�o', 'não',\
                               'transpar�ncia','transparência')[Dados], Dados)"))

#display(df_dados)

In [0]:
df_clean = df_dados.withColumn('Dados',\
                               F.expr("coalesce(map(\
                               'Dificuldade de reclamar ou de obter tratamento a reclama��o sobre dados pessoais ou financeiros',\
                               'Dificuldade de reclamar ou de obter tratamento a reclamação sobre dados pessoais ou financeiros',\
                               'Coleta, uso ou compartilhamento indevido ou n�o autorizado de dados pessoais ou financeiros',\
                               'Coleta, uso ou compartilhamento indevido ou não autorizado de dados pessoais ou financeiros',\
                               'Dados Sens�veis - Coleta, uso ou compartilhamento indevido / n�o autorizado de dado sobre origem racial, �tnica, convic��o religiosa, filos�fica, pol�tica, sa�de ou vida sexual, gen�tico ou biom�trico',\
                               'Dados Sensíveis - Coleta, uso ou compartilhamento indevido / não autorizado de dado sobre origem racial, étnica, convicãoo religiosa, filosófica, política, saúde ou vida sexual, genético ou biométrico',\
                               'Dados de Crian�as e Adolescentes - Coleta, uso ou compartilhamento indevido / n�o autorizado de dados',\
                               'Dados de Crianças e Adolescentes - Coleta, uso ou compartilhamento indevido / não autorizado de dados',\
                               'Dificuldade de cancelamento / elimina��o de dados / revoga��o do consentimento',\
                               'Dificuldade de cancelamento / eliminação de dados / revogação do consentimento',\
                               'Dificuldade para atualiza��o cadastral ou para acesso aos dados cadastrados/ Altera��o de Vencimento',\
                               'Dificuldade para atualização cadastral ou para acesso aos dados cadastrados/ Alteração de Vencimento',\
                               'Falta de clareza / transpar�ncia no uso dos dados pessoais ou financeiros (falta de informa��o sobre a finalidade, per�odo de armazenamento ou compartilhamento) ',\
                               'Falta de clareza / transparência no uso dos dados pessoais ou financeiros (falta de informação sobre a finalidade, período de armazenamento ou compartilhamento) ',\
                               'Vazamento de dados ou outros incidentes de seguran�a (acessos n�o autorizados e situa��es acidentais ou il�citas de destrui��o, perda, altera��o, comunica��o ou tratamento inadequado ou il�cito)',\
                               'Vazamento de dados ou outros incidentes de segurança (acessos não autorizados e situações acidentais ou ilícitas de destruição, perda, alteração, comunicação ou tratamento inadequado ou ilícito)')[Dados], Dados)"))

#display(df_clean)

In [0]:
df_date = df_clean.select(to_date("Data Abertura", "dd/MM/yyyy").alias("Data"), 'Dados')

df_date.printSchema()

#display(df_date)

root
-- Data: date (nullable = true)
-- Dados: string (nullable = true)

In [0]:
df_cdd = df_date.groupBy("Dados").count().orderBy("count")

display(df_cdd)

Dados,count
Dificuldade para atualização cadastral ou para acesso aos dados cadastrados/ Alteração de Vencimento,41
Dificuldade de acesso a dados pessoais ou financeiros (score),784


In [0]:
df_cdt = df_date.groupBy("Data").count().orderBy("Data")

display(df_cdt)

Data,count
2020-10-21,1
2020-10-22,1
2020-10-25,1
2020-10-27,1
2020-10-28,1
2020-10-29,2
2020-10-30,3
2020-11-01,3
2020-11-02,2
2020-11-03,5
